In [2]:
!pip install textstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 105 kB 7.2 MB/s 
     |████████████████████████████████| 2.0 MB 44.3 MB/s 


In [1]:
# Code to read csv file into Colaboratory:
# !pip install -U -q PyDrive #<-run once
# !pip install -U -q textstat #<-run once

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from google.colab import drive
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
RANDOM_STATE = 42
import pandas as pd
import nltk
import textstat
from nltk.tokenize import word_tokenize,sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from itertools import chain
import scipy as sp
import numpy as np
import time as tm
import os
import re
import math
nltk.download('averaged_perceptron_tagger') #<- run once
# nltk.download('omw-1.4') #<- run once
nltk.download('wordnet') #<- run once
nltk.download('punkt') #<- run once
# nltk.download('stopwords') #<- run once

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
# full
# files = [
#     ['1I_ij6JC8WbdP1Mcu4qhlj1m4VHIBAmVW','news.csv'],
#     ['13FxAOQIq28F0RrXSPgFfNEO-wlutAGaT', 'covid.csv'],
#     ['1LIXxaFe_l7o6xB5DEX387i7txHuKewT2', 'email.csv'],
#     ['1--9yAKlKZvqm-FB6M6Fwjgia-9fmgiGt', 'imdb.csv'],
#     ['1-0XbBZZQS6i6Iiwj2IxaIu-nM6r_BlFL', 'review.csv'],
#     ['1-2e5XJa2MkG98S03WVynPjszFKMpH1Xd', 'twitter.csv'],
#     ['1-48SJqcjlptWsxQkpLhts4J4qWBlQ8Q3', 'reddit.csv'],
#     ['1-03qp_JbkztYZFY2CXLa63a5OW4QNI6W', 'finance.csv'],
#     ['1--Mm5WfgxopRiWpHAZYgUD58eyNWneUi', 'paper.csv']
#     ]

files = [
    ['1vKAwkMHB44mUGIJl66lbhKqFKJ3gkdJA','news.csv'],
    ['1-A1oOQDNjkWFhqyAQp2QrWZRNvv8qgI1', 'covid.csv'],
    ['1-3vM3D8ERP1Vf1MSL6azPkAC9c3bQ_6H', 'email.csv'],
    ['1-3Ex8eXMs4dQq4ecGDlZ63J3QouPILkd', 'imdb.csv'],
    ['1-3ALcG0jlqkF_cDexp__OMRLPx2NKlpk', 'review.csv'],
    ['1--rSOEtY4bW_p69lhKjP5egurcKQK7-z', 'twitter.csv'],
    ['135gADn2a7vVd7-5kv8gfQIcwIStSdwx1', 'reddit.csv'],
    ['1xQgwRxzQscKyW9XMbeKA8LQbon6H7uZZ', 'finance.csv'],
    ['1ZJyXiZpyb0Lt66_Q-pVop4EuA_272XqZ', 'paper.csv']
    ]

for fl in files:
  downloaded = drive.CreateFile({'id':fl[0]}) 
  downloaded.GetContentFile(fl[1])
  tm.sleep(1)

## Meta Learning

In [5]:
# news_df = pd.read_csv("news.csv").dropna()
# covid_df = pd.read_csv("covid.csv").dropna()
# email_df = pd.read_csv("email.csv").dropna()
# imdb_df = pd.read_csv("imdb.csv").dropna()
# review_df = pd.read_csv("review.csv").dropna()
# twitter_df = pd.read_csv("twitter.csv").dropna()
# reddit_df = pd.read_csv("reddit.csv").dropna()
# finance_df = pd.read_csv("finance.csv").dropna()
# paper_df = pd.read_csv("paper.csv").dropna()

news_df = pd.read_csv("news.csv").dropna()[:1000]
covid_df = pd.read_csv("covid.csv").dropna()[:1000]
email_df = pd.read_csv("email.csv").dropna()[:1000]
imdb_df = pd.read_csv("imdb.csv").dropna()[:1000]
review_df = pd.read_csv("review.csv").dropna()[:1000]
twitter_df = pd.read_csv("twitter.csv").dropna()[:1000]
reddit_df = pd.read_csv("reddit.csv").dropna()[:1000]
finance_df = pd.read_csv("finance.csv").dropna()[:1000]
paper_df = pd.read_csv("paper.csv").dropna()[:1000]


dfs = [news_df, covid_df, email_df, imdb_df, review_df, twitter_df, reddit_df, finance_df, paper_df]

### General MF

In [6]:
dataset = [x[1].split(".")[0] for x in files]
num_of_docs = [len(d.index) for d in dfs]
num_of_sent = [len(d["sentiment"].unique()) for d in dfs]

general_df = pd.DataFrame(list(zip(dataset, num_of_docs, num_of_sent)),
               columns =['dataset', 'num_of_docs', 'num_of_sent'])

### Corpus Hardness

In [7]:
def svb_fun(dfx):
  C = dfx["sentiment"].unique()
  k = len(C)
  VCi = []
  for i in C:
    res = set()
    dfx[dfx["sentiment"] == i]["sw_exclude"].str.split(" ").apply(res.update)
    VCi.append(len(res))
  resx = set()
  dfx["sw_exclude"].str.split(" ").apply(resx.update)
  VD = len(resx)
  D = len(dfx.index)
  svb = math.sqrt(1/k * (sum(np.square((np.array(VCi) - VD)/D))))
  
  return svb

def uvb_fun(dfx):
  D = len(dfx.index)
  n = D
  Vdi = np.array(dfx["sw_exclude"].str.split(" ").apply(lambda x: len(set(x))))
  
  resx = set()
  dfx["sw_exclude"].str.split(" ").apply(resx.update)
  VD = len(resx)

  uvb = math.sqrt(1/n * (sum(np.square((np.array(Vdi) - VD)/D))))
  
  return uvb

def sem_fun(dfx):
  vct = CountVectorizer()
  tt = [dfx["sw_exclude"].str.cat(sep=" ")]
  tx = vct.fit_transform(tt)

  tf = np.array(tx.toarray()[0])
  PtiD = tf/sum(tf)

  i = len(tf)
  resx = set()
  dfx["sw_exclude"].str.split(" ").apply(resx.update)
  VD = len(resx)

  QtiD = (1/i)/sum(1/np.arange(1,VD+1))

  sem = sum(PtiD * np.log(PtiD/QtiD))
  return sem

def short_fun(dfx):
  n = len(dfx.index)
  di = np.array(dfx['sw_exclude'].str.split(" ").apply(len))
  DL = (1/n) * sum(di)

  Vdi = np.array(dfx["sw_exclude"].str.split(" ").apply(lambda x: len(set(x))))  
  VL = (1/n) * sum(Vdi)

  VDR = np.log(VL)/np.log(DL)

  # average word length
  AWL = np.mean([len(i.strip()) for i in dfx["sw_exclude"].str.cat(sep=" ").split(" ")])  

  return {'DL': DL, 'VL': VL, 'VDR': VDR, 'AWL': AWL}

In [8]:
sht = [short_fun(i) for i in dfs]
hardness_df = pd.DataFrame(list(zip(
    dataset, 
    [svb_fun(i) for i in dfs], 
    [uvb_fun(i) for i in dfs],
    [sem_fun(i) for i in dfs],
    [i['VL'] for i in sht],
    [i['VDR'] for i in sht],
    [i['AWL'] for i in sht]
    )),
    columns =['dataset',
              'svb',
              'uvb',
              'sem',
              'vl',
              'vdr',
              'awl'
    ])

### Corpus Readability

In [9]:
def rdblty(dfx):
  tt = dfx['sw_exclude'].str.cat(sep='. ')
  fre = textstat.flesch_reading_ease(tt)
  smog = textstat.smog_index(tt)
  fkg = textstat.flesch_kincaid_grade(tt)
  cli = textstat.coleman_liau_index(tt)
  ari = textstat.automated_readability_index(tt)
  dcr = textstat.dale_chall_readability_score(tt)
  lw = textstat.linsear_write_formula(tt)
  gf = textstat.gunning_fog(tt)

  return [fre, smog, fkg, cli, ari, dcr, lw, gf]

In [10]:
data1 = pd.DataFrame(dataset, columns=['dataset'])
data2 = pd.DataFrame([rdblty(i) for i in dfs], columns=['flesch_reading_ease', 
                                                    'smog', 
                                                    'flesch_kincaid_grade', 
                                                    'coleman_liau_index', 
                                                    'auto_readability_index', 
                                                    'dale_chall_readability',
                                                    'linsear_write',
                                                    'gunning_fog'])
rdblty_df = pd.concat([data1,data2], axis=1)

### Statistical Features

In [11]:
def word_dist(dfx):
  tt = dfx['sw_exclude'].str.split(" ").apply(len)
  min_ = min(tt)
  max_ = max(tt)
  mean = np.mean(tt)
  std_ = np.std(tt)
  skw = sp.stats.skew(tt)
  krt = sp.stats.kurtosis(tt)
  mean_std_ratio = mean/std_

  val_count = np.array(dfx['sentiment'].value_counts())
  ent = sp.stats.entropy(val_count/sum(val_count))
  return [min_, max_, mean, std_, skw, krt, mean_std_ratio, ent]

In [12]:
data3 = pd.DataFrame([word_dist(i) for i in dfs], columns=['min', 
                                                    'max', 
                                                    'mean', 
                                                    'std', 
                                                    'skew', 
                                                    'kurtosis',
                                                    'mean_std_ratio',
                                                    'entropy'])
word_dist_df = pd.concat([data1,data3], axis=1)

### Lexical Features

In [13]:
def lx(l):
  return nltk.pos_tag(nltk.word_tokenize(l))

def lex(dfx):
  tt = dfx['sw_include'].str.lower()
  lxx = np.array(list(map(lx, tt)))
  pos = [j[1] for j in list(chain(*[i for i in lxx]))]

  pos_mean = pd.Series(pos).value_counts()
  pos_tuple = list(zip(pos_mean.index,pos_mean))

  adj = 0
  adp = 0
  adv = 0
  conj = 0
  det = 0
  nn = 0
  num = 0
  part = 0
  pron = 0
  vrb = 0
  oth =  0

  for tup in pos_tuple:
    ttx = tup[0]
    if ttx.startswith('JJ'):
      adj+=tup[1]
    elif ttx.startswith('RB'):
      adv+=tup[1]
    elif ttx.startswith('NN'):
      nn+=tup[1]
    elif ttx.startswith('PRP'):
      pron+=tup[1]
    elif ttx.startswith('VB'):
      vrb+=tup[1]
    elif ttx == 'IN':
      adp+=tup[1]
    elif ttx == 'CC':
      conj+=tup[1]
    elif ttx == 'DT':
      det+=tup[1]
    elif ttx == 'CD':
      num+=tup[1]
    elif ttx == 'RP':
      part+=tup[1]
    else:
      oth+=tup[1]

  pos_fin = np.array([adj,adp,adv,conj,det,nn,num,part,pron,vrb,oth])/len(tt)
  
  return pos_fin

In [14]:
data4 = pd.DataFrame([lex(i) for i in dfs], columns=['adjective', 
                                                    'adposition', 
                                                    'adverb', 
                                                    'conjunction', 
                                                    'determiner', 
                                                    'noun',
                                                    'numeral',
                                                    'particle',
                                                    'pronoun',
                                                    'verb',
                                                    'other'])
lex_df = pd.concat([data1,data4], axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


### To CSV

In [18]:
general_df.to_csv(wd+"mlr_general.csv", index=False)
hardness_df.to_csv(wd+"mlr_hardness.csv", index=False)
rdblty_df.to_csv(wd+"mlr_rdblty.csv", index=False)
word_dist_df.to_csv(wd+"mlr_word_dist.csv", index=False)
lex_df.to_csv(wd+"mlr_lex.csv", index=False)

In [17]:
merged_df = general_df\
.join(hardness_df.set_index('dataset'), on='dataset', how='left')\
.join(rdblty_df.set_index('dataset'), on='dataset', how='left')\
.join(word_dist_df.set_index('dataset'), on='dataset', how='left')\
.join(lex_df.set_index('dataset'), on='dataset', how='left')

wd = '/content/drive/MyDrive/UNI/COMPSCI 760/assignments/group project/scripts/output/'
merged_df.to_csv(wd+"meta_learning_full.csv", index=False)